# Lab 10 -- Decision Trees for Regression

Compared to last week, this is a very simple lab <span style="font-size:20pt;">😃</span> You'll have fun programming!

You will implement the **Classification and Regression Tree (CART)** algorithm from scratch.

The lab is broken down into the following pieces:

* Regression Criterion
* Creating Splits
* Buiding a Tree
* Making a prediction

## Exercise 1 -- Download and load the dataset

We will be using the usual Boston Housing dataset, which is available to download from ECLASS

* Download the file
* Read it and separate the target variable from the features.
* Make a 80/10/10 train/validation/test split

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# your code here
boston = pd.read_csv("datasets/BostonHousing.txt")

The target variable will be as usual `MEDV`. Use the rest as features.

In [3]:
# your code here
X = boston.iloc[:,0:13].to_numpy()
y = boston.iloc[:,13].to_numpy()
X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.2,random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,test_size=0.5, random_state = 42)

## Exercise 2 -- Optimization Criterion

For regression, a simple criterion to optimize is to minimize the sum of squared errors for a given region. This is, for all datapoints in a region with size, we minimize:

$$\sum_{i=1}^N(y_i - \hat{y})^2$$

where $N$ is the number of datapoits in the region and $\hat{y}$ is the mean value of the region for the target variable. 

Implement such a function using the description below.

Please, don't use an existing implementation, refer to the [book](https://www.statlearning.com/s/ISLRSeventhPrinting.pdf), and if you need help, ask questions!

In [4]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity (use: float("inf"))
    This avoids creating empty regions
    """
    if len(region) < 1:
        return float("inf")
    mean = np.mean(region)
    target = region.reshape(-1,1)
    sse = np.sum(np.square(target-mean))
    
    return sse
    # your code here

In [5]:
rng = np.random.default_rng(0)
print(regression_criterion(rng.random(size=40)))
print(regression_criterion(np.ones(10)))
print(regression_criterion(np.zeros(10)))
print(regression_criterion(np.array([])))

3.6200679838629544
0.0
0.0
inf


## Exercise 3 -- Make a split

In [6]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    left_partition : array
        indices of the datapoints in `region` where feature < `tau`
    right_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    # your code here
    left_partition = []
    right_partition = []
    for i in range(region.shape[0]):
        if region[i, feature_index] < tau:
            left_partition.append(i)
        else:
            right_partition.append(i)
    
    return np.array(left_partition), np.array(right_partition)

In [7]:
l, r = split_region(X_train, 2, 10)
print(l.shape, r.shape)

(221,) (183,)


## Exercise 4 -- Find the best split

The strategy is quite simple (as well as inefficient), but it helps to reinforce the concepts.
We are going to use a greedy, exhaustive algorithm to select splits, selecting the `feature_index` and the `tau` that minimizes the Regression Criterion

In [8]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'right_region' -> indices not in `low_region`
    """
    n = X.shape[0]
    m = X.shape[1]
    decision = {'feature_index': 0,
                 'tau':0,
                 'left_region':0,
                 'right_region':0}
    lowest_criterion = float("inf")
    for j in range(m):
        for i in range(n):
            tau = X[i,j]
            l, r = split_region(X, j, tau)
            if len(l) >= 1 and len(r) >= 1:
                sum_criterion = regression_criterion(y[l]) + regression_criterion(y[r])
            if len(r) < 1: 
                sum_criterion = regression_criterion(y[l]) + regression_criterion([])
            if len(l) < 1:
                sum_criterion = regression_criterion(y[r]) + regression_criterion([])
 
            if sum_criterion < lowest_criterion:
                lowest_criterion = sum_criterion    
                decision['feature_index'] = j
                decision['tau'] = tau
                decision['left_region'] = l
                decision['right_region'] = r
            
    return decision        

In [9]:
get_split(X_train[:15, :], y_train[:15])

{'feature_index': 4,
 'tau': 0.614,
 'left_region': array([ 1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 14]),
 'right_region': array([ 0,  3,  8, 13])}

## Exercise 5 -- Recursive Splitting

The test above is an example on how to find the root node of our decision tree. The algorithm now is a greedy search until we reach a stop criterion. To find the actual root node of our decision tree, you must provide the whole training set, not just a slice of 15 rows as the test above.

The trivial stopping criterion is to recursively grow the tree until each split contains a single point (perfect node purity). If we go that far, it normally means we are overfitting.

You will implement these criteria to stop the growth:

* A node is a leaf if:
    * It has less than `min_samples` datapoints
    * It is at the `max_depth` level from the root (each split creates a new level)
    * The criterion is `0`



In [28]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
    """
    #verifica se é node terminal
    if 'left_region' not in node or 'right_region' not in node:
        return
    
    #pega os indices do datapoint
    left_indices = node['left_region']
    right_indices = node['right_region']
    
    #verifica criterio de parada pro node left
    if len(left_indices) <= min_samples or current_depth >= max_depth:
        node['left'] = {'value': np.mean(y[left_indices])}
    # faz partição do left 
    else:
        node['left'] = get_split(X[left_indices], y[left_indices])
        recursive_growth(node['left'], min_samples, max_depth, current_depth + 1, X[left_indices], y[left_indices])
    
    #verifica criterio de parada pro node right
    if len(right_indices) <= min_samples or current_depth >= max_depth:
        node['right'] = {'value': np.mean(y[right_indices])}
    # faz partição do right
    else:
        node['right'] = get_split(X[right_indices], y[right_indices])
        recursive_growth(node['right'], min_samples, max_depth, current_depth + 1, X[right_indices], y[right_indices])
    

In [29]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 3, 1, X_train[:k, :], y_train[:k])

Below we provide code to visualise the generated tree!

In [30]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)


In [31]:
print_tree(test_root, 0)

 X_9 < 224.0
 [38.0]
.   X_5 < 6.383
.  .   X_0 < 0.84054
.  .   [19.814285714285713]
.  .   [11.94]
.  .   X_5 < 7.163
.  .   [24.98]
.  .   [31.6]


# Exercise 6 -- Make a Prediction
Use the a node to predict the class of a compatible dataset

In [32]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    while 'value' not in node:
        if sample[node['feature_index']] >= node['tau']:
            node = node['right']
        else:
            node = node['left']
    
    return node['value']
    # your code here
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    y_pred = []
    for x in X:
        pred = predict_sample(node,x)
        y_pred.append(pred)
    
    return np.array(y_pred)
    # your code here

Now use the functions defined above to calculate the RMSE of the validation set. 
* Try first with `min_samples=20` and `max_depth=6` (for this values you should get a validation RMSE of ~8.8)

Then, experiment with different values for the stopping criteria.

In [33]:
# your code here
min_samples = 20
max_depth = 6
root = get_split(X_train,y_train)
recursive_growth(root,min_samples,max_depth,1, X_train, y_train)
y_pred = predict(root,X_val)

In [34]:
rmse = np.sqrt(np.mean(np.square(y_val-y_pred)))

In [35]:
rmse

3.0161878077187203

In [36]:
print_tree(root, 0)

 X_5 < 6.943
.   X_12 < 14.43
.  .   X_7 < 1.413
.  .   [44.475]
.  .  .   X_5 < 6.546
.  .  .  .   X_5 < 6.101
.  .  .  .  .   X_8 < 2.0
.  .  .  .  .   [16.133333333333333]
.  .  .  .  .   [20.539130434782606]
.  .  .  .  .   X_9 < 223.0
.  .  .  .  .   [31.5]
.  .  .  .  .   [22.401234567901234]
.  .  .  .   X_12 < 5.89
.  .  .  .   [30.05]
.  .  .  .  .   X_7 < 4.148
.  .  .  .  .   [27.599999999999998]
.  .  .  .  .   [24.693333333333335]
.  .   X_0 < 7.05042
.  .  .   X_6 < 73.5
.  .  .   [21.220000000000002]
.  .  .  .   X_0 < 0.17171
.  .  .  .   [19.733333333333334]
.  .  .  .  .   X_6 < 94.0
.  .  .  .  .   [17.15]
.  .  .  .  .   [15.00909090909091]
.  .  .   X_12 < 19.92
.  .  .  .   X_5 < 6.852
.  .  .  .   [14.135]
.  .  .  .   [27.5]
.  .  .  .   X_4 < 0.679
.  .  .  .   [13.170000000000002]
.  .  .  .  .   X_0 < 11.8123
.  .  .  .  .   [10.666666666666666]
.  .  .  .  .   [8.033333333333335]
.   X_5 < 7.454
.  .   X_0 < 8.24809
.  .  .   X_7 < 1.8946
.  .  .   [45.65]
.